In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import warnings
warnings.filterwarnings("ignore")
import os
import string
import re
#nltk models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score, cohen_kappa_score, roc_auc_score
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#classifiers
import sklearn
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

import keras
from keras.preprocessing import sequence
from keras.layers import Dropout, Dense, Flatten, Embedding,Activation
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D 
from keras.layers import GlobalMaxPooling1D
from sklearn.svm import LinearSVC
np.random.seed(0)

In [2]:
os.chdir(r'C:\Users\oyo\Desktop')
df = pd.read_excel(r'data_set.xlsx',encoding = 'ISO-8859-1')
df.head()

,Review,AC/Heater,L2 AC/Heater,Check-in Experience,L2 Check-in Experience,Comfort & Safety,L2 Comfort & Safety,Food Experience,L2 Food Experience,Hotel Infrastructure,...,L2 Hygiene & Cleanliness,Room Equipment & Amenities,L2 Room Equipment & Amenities,Staff & Service,L2 Staff & Service,TV & WiFi,L2 TV & WiFi,Washroom,L2 Washroom,Unnamed: 21
0,quiet friendly staff and spacious room. housek...,0,4,0,5,0,6,0,6,0,...,5,0,5,0,4,0,7,0,8,NaN
1,Stayed at this hotel a few times and always ta...,0,4,1,0,0,6,0,6,0,...,5,0,5,0,4,0,7,0,8,NaN
2,"We were planning a really late check in, so I ...",0,4,0,5,0,6,0,6,0,...,5,0,5,0,4,0,7,0,8,NaN
3,Renovating rooms right now . good beds and nic...,0,4,0,5,0,6,0,6,0,...,5,0,5,0,4,0,7,0,8,NaN
4,"Overall I'd give it 4 stars, newly remodeled r...",0,4,0,5,0,6,0,6,0,...,5,0,5,0,4,0,7,0,8,NaN


In [3]:
#for L1
stemmer2 = SnowballStemmer('english')
## preprocessing text:
stop_words = set(stopwords.words('english'))
filter_SW = {"very","until","out","than",'ain','against','aren',"aren't","arent",'couldn',"couldn't","couldnt","didn","didn't","didnt","doesn","doesn't","doesnt","don","don't","dont","hadn","hadn't","hadnt","hasn","hasn't","haven","haven't","hasnt","isn","isn't","isnt","mightn't","mightnt","mightn","mustn","mustn't","mustnt","needn","needn't","neednt","no","not","nor","off","shan","shan't","shant","shouldn","shouldn't","shouldnt","wasn't","wasnt","wasn","weren","weren't","werent","won't","wont","won","wouldn","wouldn't","wouldnt"}
Nstop_words = stop_words - filter_SW
Nstop_words.update(["'","' "," '","’"," ’","’ "])
punct = ""
for i in string.punctuation:
    if(i!="'" ):
        punct = punct+i
#print(punct)

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
def preprocessing_text(text,lemma=True):
    text = text.lower()
    #replacing apostrophe with none
    text = text.replace("’","")
    text = text.replace("'","")
    text = text.replace('/','')
    #replacing special characters with a space
    text = re.sub('[^A-Za-z]+', " ", text)
    
    #replacing newline,tabs with none
    text = re.sub(r"[\n\t]*", "", text)
    
    #removing multiple sapces
    text = re.sub(" +"," ", text)
    
    #removing punctuations except apostrophe
    text = [word.strip(punct) for word in text.split(" ")]
    
    # remove words that contain numbers
    #text = [word for word in text if not any(c.isdigit() for c in word)]
    
    #removing stopwords
    text = [x for x in text if x not in Nstop_words]
    
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    
    # lemmatize text
    if lemma==True:
        pos_tags = pos_tag(text)
        text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
        
    else:
        ps = PorterStemmer()
        text =  [stemmer2.stem(word) for word in text]
        
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    
    return(text)

In [4]:
stem=[]
for i in range(len(df)):
    t=preprocessing_text(str(df['Review'][i]),lemma=False)#false for stemming and true for lemmatization
    stem.append(t)

In [5]:
#sequence padding

tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(stem)
sequences = tokenizer.texts_to_sequences(stem)
seq = sequence.pad_sequences(sequences , maxlen = 60,padding = 'post', truncating = 'post' )
#print(tokenizer.word_counts)
#print("shape: ",np.shape(seq))
#print(seq)


In [6]:
#creating the model
reshape = keras.layers.Reshape
model = Sequential()
model.add(Embedding(input_dim = 2000, output_dim = 10 , input_length = 60))
model.add(Dropout(0.1))
model.add(Conv1D(64,kernel_size=3, strides = 1,activation='relu', padding = 'valid'))
model.add(MaxPooling1D(pool_size = 2, padding = 'same'))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 10)            20000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 10)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 58, 64)            1984      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 29, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 27, 32)            6176      
_________________________________________________________________
global_max_pooling1d_

In [13]:
#importing text file
Data = pd.read_excel(r'test_set.xlsx', encoding = 'ISO-8859-1')
#removing highly positive comments
model = SentimentIntensityAnalyzer()
sentiment = pd.DataFrame(columns=['neg','neu','pos','compound'])
for i in range(len(Data)):
    
    sentence = Data['Review'][i]
    sentence = str(sentence)
    ss = model.polarity_scores(sentence)
    sentiment =  sentiment.append(ss , ignore_index = True)
DF = pd.concat([Data,sentiment], axis = 1)
DF['sentiment']=DF['compound'].apply(lambda x: 0  if x<0.9 else np.nan )
#result.head(10)
nan_rows = DF[DF['sentiment'].isnull()].index.values
DF = DF.drop(nan_rows)
DF.reset_index()
print(len(DF))
print(len(Data))

125
127


In [8]:
#sequnce padding of test file
com=[]
for i in range(len(DF)):
    t=preprocessing_text(DF['Review'][i], lemma = False)
    com.append(t)
seqcs = tokenizer.texts_to_sequences(com)
seq1 = sequence.pad_sequences(seqcs , maxlen = 60)

In [9]:
#creating the output file of L1 tagging
out=pd.DataFrame(columns=['AC/Heater_Predict','Check-in Experience_Predict', 'Comfort & Safety_Predict', 'Food Experience_Predict', 'Hotel Infrastructure_Predict','Hygiene & Cleanliness_Predict','Room Equipment & Amenities_Predict','Staff & Service_Predict','TV & WiFi_Predict','Washroom_Predict'])
output_L1 = pd.concat([DF, out], axis=1)
output_L1.head()

,Review,AC/Heater_Predict,Check-in Experience_Predict,Comfort & Safety_Predict,Food Experience_Predict,Hotel Infrastructure_Predict,Hygiene & Cleanliness_Predict,Room Equipment & Amenities_Predict,Staff & Service_Predict,TV & WiFi_Predict,Washroom_Predict
0,good location,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,THA HOYEL IS NEW UNDRER WORKING BUT EXLENT FRE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ROOM NEAT AND CLEAN BUT WATER IS NOT GOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bad experience. No hotwater. Breakfast not ser...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,room quality is not good. good for couples,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#add other classes
dict_labels={'y1':'AC/Heater',
'y2':'Check-in Experience',
'y3':'Comfort & Safety',
'y4':'Food Experience',
'y5':'Hotel Infrastructure',
'y6':'Hygiene & Cleanliness',
'y7':'Room Equipment & Amenities',
'y8':'Staff & Service',
'y9':'TV & WiFi',
'y10':'Washroom',
}

dict_labels2={'y1':'AC/Heater_Predict',
'y2':'Check-in Experience_Predict',
'y3':'Comfort & Safety_Predict',
'y4':'Food Experience_Predict',
'y5':'Hotel Infrastructure_Predict',
'y6':'Hygiene & Cleanliness_Predict',
'y7':'Room Equipment & Amenities_Predict',
'y8':'Staff & Service_Predict',
'y9':'TV & WiFi_Predict',
'y10':'Washroom_Predict',
}

for i in dict_labels:
    print("-------"+dict_labels[i]+"-------")
    X_train,X_test,Y_train,Y_test = train_test_split(seq,df[dict_labels[i]], test_size = 0.05)
    x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.08)
    model.fit(x_train, y_train,batch_size = 32, validation_data=(x_val, y_val), class_weight = 'balanced', epochs=5)
    #predicting results for test set
    x_test = seq1
    y_sc1= model.predict(x_test)
    y_score1 = np.zeros(len(y_sc1))
    for j in range(len(y_score1)):
        if y_sc1[j] <= 0.51:
            y_score1[j] = 0
        else:
            y_score1[j] = 1
    output_L1[dict_labels2[i]] = y_score1
    
    #calculating accuracy with standard probability distribution
    y_sc = model.predict(X_test)
    y_score = np.zeros(len(y_sc))
    for i in range(len(y_sc)):
        if y_sc[i] < 0.51:
            y_score[i] = 0
        else:
            y_score[i] = 1
            
    
    print("Accuracy score of L1:  {} " .format(str(accuracy_score(Y_test,y_score))))
    print("roc_auc score of L1 model:{} ".format(str(roc_auc_score(Y_test, y_sc))))
    print(confusion_matrix(Y_test,y_score))  
    print(classification_report(Y_test,y_score))

-------AC/Heater-------
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 16623 samples, validate on 1446 samples
Epoch 1/5
16623/16623 [==============================] - 5s 293us/step - loss: 0.1889 - acc: 0.9586 - val_loss: 0.1344 - val_acc: 0.9703
Epoch 2/5
16623/16623 [==============================] - 4s 270us/step - loss: 0.1403 - acc: 0.9626 - val_loss: 0.0767 - val_acc: 0.9730
Epoch 3/5
16623/16623 [==============================] - 4s 248us/step - loss: 0.0694 - acc: 0.9735 - val_loss: 0.0635 - val_acc: 0.9765
Epoch 4/5
16623/16623 [==============================] - 4s 267us/step - loss: 0.0530 - acc: 0.9795 - val_loss: 0.0613 - val_acc: 0.9758
Epoch 5/5
16623/16623 [==============================] - 4s 265us/step - loss: 0.0425 - acc: 0.9847 - val_loss: 0.0658 - val_acc: 0.9786
Accuracy score of L1:  0.9747634069400631 
roc_auc score of L1 model:0.9511835268458528 
[[908   9]
 [ 15  19]]
    

KeyboardInterrupt: 

In [ ]:
output_L1.reset_index()
output_L1.head()

In [ ]:
t_Vector = TfidfVectorizer(max_features=2000, min_df=0.001, max_df=0.5,stop_words=Nstop_words,ngram_range = (1,4))
#x = t_Vector.fit_transform(stem)


In [ ]:
df_cols = df[['L2 AC/Heater', 'L2 Check-in Experience',
       'L2 Comfort & Safety', 'L2 Food Experience', 'L2 Hotel Infrastructure',
       'L2 Hygiene & Cleanliness', 'L2 Room Equipment & Amenities', 'L2 Staff & Service',
       'L2 TV & WiFi', 'L2 Washroom']]
dict_labels = {'y1':'L2 AC/Heater',
'y2':'L2 Check-in Experience',
'y3':'L2 Comfort & Safety',
'y4':'L2 Food Experience',
'y5':'L2 Hotel Infrastructure',
'y6':'L2 Hygiene & Cleanliness',
'y7':'L2 Room Equipment & Amenities',
'y8':'L2 Staff & Service',
'y9':'L2 TV & WiFi',
'y10':'L2 Washroom',
}
for i in dict_labels:
    text = df_cols[dict_labels[i]]
    encode = LabelEncoder()
    df[dict_labels[i]] = encode.fit_transform(text.astype(str))

In [ ]:
outt=pd.DataFrame(columns=['L2 AC/Heater predict','L2 Check-in Experience predict', 'L2 Comfort & Safety predict', 'L2 Food Experience predict', 'L2 Hotel Infrastructure predict','L2 Hygiene & Cleanliness predict','L2 Room Equipment & Amenities predict','L2 Staff & Service predict','L2 TV & WiFi predict','L2 Washroom predict'])
final = pd.concat([output_L1, outt], axis=1)
final.head()

In [ ]:
#getting predictions for L2 using rbf kernel svm
dict_labels_L1 = {'y1':'AC/Heater',
'y2':'Check-in Experience',
'y3':'Comfort & Safety',
'y4':'Food Experience',
'y5':'Hotel Infrastructure',
'y6':'Hygiene & Cleanliness',
'y7':'Room Equipment & Amenities',
'y8':'Staff & Service',
'y9':'TV & WiFi',
'y10':'Washroom',
}
dict_labels_L2 = {'y1':'L2 AC/Heater',
'y2':'L2 Check-in Experience',
'y3':'L2 Comfort & Safety',
'y4':'L2 Food Experience',
'y5':'L2 Hotel Infrastructure',
'y6':'L2 Hygiene & Cleanliness',
'y7':'L2 Room Equipment & Amenities',
'y8':'L2 Staff & Service',
'y9':'L2 TV & WiFi',
'y10':'L2 Washroom',
}
dict_labels2={'y1':'AC/Heater_Predict',
'y2':'Check-in Experience_Predict',
'y3':'Comfort & Safety_Predict',
'y4':'Food Experience_Predict',
'y5':'Hotel Infrastructure_Predict',
'y6':'Hygiene & Cleanliness_Predict',
'y7':'Room Equipment & Amenities_Predict',
'y8':'Staff & Service_Predict',
'y9':'TV & WiFi_Predict',
'y10':'Washroom_Predict',
}
dict_labels_L3 = {'y1':'L2 AC/Heater predict',
'y2':'L2 Check-in Experience predict',
'y3':'L2 Comfort & Safety predict',
'y4':'L2 Food Experience predict',
'y5':'L2 Hotel Infrastructure predict',
'y6':'L2 Hygiene & Cleanliness predict',
'y7':'L2 Room Equipment & Amenities predict',
'y8':'L2 Staff & Service predict',
'y9':'L2 TV & WiFi predict',
'y10':'L2 Washroom predict',
}
result = pd.DataFrame()

for i in dict_labels_L1:

    print("-------"+dict_labels_L1[i]+"-------")
    df1  = pd.DataFrame()
    DF1 = pd.DataFrame()
    df1 = df1.append(df[df[dict_labels_L1[i]] == 1])
    DF1 = DF1.append(final[final[dict_labels2[i]] == 1])
    df1 = df1.reset_index()
    DF1 = DF1.reset_index()
    #print(len(DF1))
    y = df1[dict_labels_L2[i]]
    #y1 = DF1[dict_labels_L2[i]]
    
    #X,Y = nr.fit_sample(df1['Review'],y)
    stem=[]
    stem1 = []
    for j in range(len(df1)):
        t = str(df1['Review'][j])
        t = preprocessing_text(t,lemma = False) 
        stem.append(t)
        
    if len(DF1) != 0:
        for j in range(len(DF1)):
            t1 = str(DF1['Review'][j])
            t1 = preprocessing_text(t1, lemma = False)
            stem1.append(t1)
        '''
        embeddings = embed(stem)
        embeddings1 = embed(stem1)
        config = tf.ConfigProto()
        config.graph_options.rewrite_options.shape_optimization = 2
        session = tf.Session(config=config)
        sess = session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        x = session.run(embeddings)   
        x1 = session.run(embeddings1)
        '''
        x = t_Vector.fit_transform(stem)
        x_train,x_test,y_train,Y_test = train_test_split(x,y,test_size=0.15,random_state = 20)
        model = RFE(RandomForestClassifier(max_features = 'auto', criterion = 'gini', bootstrap = True,class_weight = 'balanced'), 50, step=10)
        model.fit(x_train,y_train)
        x1 = t_Vector.transform(stem1)

        '''
        #x_test,Y_test = (x1,y1)
        #clf = RandomForestClassifier(class_weight = 'balanced')
        #svm = LinearSVC(multi_class = 'ovr' , class_weight = 'balanced' )
        clf = SVC(kernel = 'rbf',class_weight = 'balanced', decision_function_shape = 'ovr', C = 1000000,gamma = 1e-06 )
        #clf = CalibratedClassifierCV(svm)
        clf.fit(x_train,y_train)
        #model = MultinomialNB(alpha=1, class_prior=None, fit_prior=True).fit(x_train,y_train)
        '''
        y_score1 = model.predict(x1)
        y_score = model.predict(x_test)
        DF1[dict_labels_L3[i]] = y_score1 
        result = result.append(DF1)

        print("Balanced_accuracy score is {}\n".format(str(balanced_accuracy_score(Y_test, y_score))))
        print("accuracy score: ", accuracy_score(Y_test,y_score))
        print("Cohen kappa score : ", cohen_kappa_score(Y_test, y_score))
        print(confusion_matrix(Y_test,y_score))  
        print(classification_report(Y_test,y_score))
    

In [ ]:
result.head()


In [ ]:
result.to_csv("result_file.csv", encoding = 'utf-8', index = False)